In [1]:
import cv2
import numpy as np
import tensorflow as tf
import pickle
import tempfile

In [2]:
# Function to load video and mask, and get parking spot bounding boxes
def load_video_and_mask(video_file, mask_file):
    # Load the video
    cap = cv2.VideoCapture(filename=video_file)
    
    # Load the mask
    mask = cv2.imread(mask_file, cv2.IMREAD_GRAYSCALE)
    
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Get bounding boxes for each contour (parking spot)
    parking_spots = [cv2.boundingRect(contour) for contour in contours]
    
    return cap, parking_spots

In [3]:
# Function to predict occupancy using the trained model
def predict_occupancy(frame, parking_spots, model, target_size):
    predictions = []
    spots = []
    for (x, y, w, h) in parking_spots:
        spot = frame[y:y+h, x:x+w]
        spot_resized = cv2.resize(spot, (model.input_shape[2], model.input_shape[1]))
        spot_resized = np.expand_dims(spot_resized, axis=0)  # Add batch dimension
        prediction = model.predict(spot_resized)
        predictions.append(prediction[0][0])
    return predictions

In [4]:
# Function to process the entire video and export with rectangles and scores
def export_video_with_predictions(video_file, mask_file, output_path, model, input_shape):
    cap, parking_spots = load_video_and_mask(video_file, mask_file)
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, frame_rate, (frame_width, frame_height))

    for frame_number in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break

        predictions = predict_occupancy(frame, parking_spots, model, input_shape)

        for i, (x, y, w, h) in enumerate(parking_spots):
            color = (0, 255, 0) if predictions[i] < 0.5 else (0, 0, 255)  # Green for empty, Red for occupied
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, f'{predictions[i]:.2f}', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        out.write(frame)

    cap.release()
    out.release()

In [6]:
# Load the trained TensorFlow model
filehandle = open('model_MobileNetV2_4.pkl','rb')
model = pickle.load(filehandle)
input_shape = model.input_shape[1:3]

In [7]:
# Upload video file
video_file = 'parking_crop.mp4'
mask_file = 'mask_crop.png'

In [8]:
temp_output = tempfile.NamedTemporaryFile(delete=False, suffix='.mp4').name
print(temp_output)
export_video_with_predictions(video_file, mask_file, temp_output, model, input_shape)

/var/folders/0r/tspq5j2s2fv0zp31f7l4m0_c0000gn/T/tmp0zufirhr.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 466ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━